In [1]:
import os
import sys
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.4-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")


## Create SparkContext, SparkSession
from pyspark.sql import SparkSession
from pyspark import SparkContext
sc = SparkContext()

In [2]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array
from math import sqrt

In [3]:
data = sc.textFile("/user/rameshm/uber/HistData/FlumeData.*")

In [4]:
data.count()

3296451

In [5]:
data.take(2)

[u'"11/1/2016 0:00:00",40.7293,-73.992,"B02512"',
 u'"11/1/2016 0:01:00",40.7131,-74.0097,"B02512"']

In [6]:
def convertDataFloat(line):
    return array([float(line[1]),float(line[2])])

In [7]:
fea_data = data.map(lambda data:data.split(','))
parsedData = fea_data.map(lambda line : convertDataFloat(line))

In [8]:
parsedData.take(5)

[array([ 40.7293, -73.992 ]),
 array([ 40.7131, -74.0097]),
 array([ 40.3461, -74.661 ]),
 array([ 40.7555, -73.9833]),
 array([ 40.688 , -74.1831])]

In [9]:
clusters = KMeans.train(parsedData,8, maxIterations=10, initializationMode="random")

In [10]:
clusters.centers

[array([ 40.85076585, -73.58599347]),
 array([ 40.77518455, -73.86576704]),
 array([ 40.68666259, -73.9635233 ]),
 array([ 40.72735938, -73.99992808]),
 array([ 40.75651621, -73.98022921]),
 array([ 40.79097987, -73.95365542]),
 array([ 40.65481718, -73.78217286]),
 array([ 40.6981853 , -74.20277296])]

In [11]:
def wsssError(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

In [12]:
WSSSE = parsedData.map(lambda point: wsssError(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

Within Set Sum of Squared Error = 64573.8835831


In [13]:
clusters.predict(array([40.6988701 , -74.20341933]))

7

In [14]:
sqrt(sum((array([40.7204,-74.0047]) - array([ 40.71743048, -74.002436  ])) ** 2))

0.0037341324334293183

In [14]:
#40.7204 - 40.71743048

In [15]:
#-74.0047 - -74.002436

In [15]:
clusters.save(sc, "/user/rameshm/Uber/kmeanModel")